# Dataset
Daatset includes various data regarding: delivery person, restaurant and deliry location, date and time, weather traffic. Dataset contains around 45.5 k deliveries. 

In [14]:
import pandas as pd
df = pd.read_csv("raw_data.csv", na_values="NaN ")
# print(len(df))

ID                                        0x4f0 
Delivery_person_ID                MUMRES17DEL02 
Delivery_person_Age                          NaN
Delivery_person_Ratings                      NaN
Restaurant_latitude                    19.121999
Restaurant_longitude                   72.908493
Delivery_location_latitude             19.201999
Delivery_location_longitude            72.988493
Order_Date                            06-04-2022
Time_Orderd                                  NaN
Time_Order_picked                       18:35:00
Weatherconditions              conditions Cloudy
Road_traffic_density                     Medium 
Vehicle_condition                              1
Type_of_order                            Drinks 
Type_of_vehicle                         scooter 
multiple_deliveries                          1.0
Festival                                     No 
City                              Metropolitian 
Time_taken(min)                         (min) 33
Name: 42, dtype: obj

# Dataset clearing
Firstly, deliveries with missing data and incorrect geographical coordinates (outside of India) were filltered and deleted. There are 4225 rows with missing values. Additionally there are 3450 rows where restaurant or delivery location are outside India. After initial preprocessing almost 38k deliveries are suitable for further processing.

In [17]:
india_bbox = (68.1766451354, 7.96553477623, 97.4025614766, 35.4940095078)
india_high_lat, india_low_lat = 35.4940095078, 7.96553477623
india_high_long, india_low_long = 97.4025614766, 68.1766451354

df.dropna(inplace=True)

print(45593-len(df))


4225


In [19]:
outside_locations_df = df.loc[(df['Restaurant_latitude'] < india_low_lat) |
                              (df['Restaurant_latitude'] > india_high_lat) |
                              (df['Restaurant_longitude'] < india_low_long) |
                              (df['Restaurant_longitude'] > india_high_long) |
                              (df['Delivery_location_latitude'] < india_low_lat) |
                              (df['Delivery_location_latitude'] > india_high_lat) |
                              (df['Delivery_location_longitude'] < india_low_long) |
                              (df['Delivery_location_longitude'] > india_high_long)]
print(len(outside_locations_df))

df.drop(outside_locations_df.index, inplace=True)
print(len(df))

0
37918


# Preprocessing using OSRM API
Next step is to map geographic coordinates onto road distance. To produse routes we used OSRM API that allows to find the fastest route between coordinates (not sure if we use route service ot trip service) via HTTP request. API is prefered as in contrast to library based clients as it produces higher quality data (less errors such impossible routes, incorrect distance etc.). Created OSRM client uses asynchnonic HTTP request to produce route biteewn restaurant and delivery location and if response is successful it save the route in csv file. Additionally logger was creates which save information about unsuccessful request and its reason. While genereting routes only one type of error occured - "Too many request". This indicates that server was reached max numer of request per minute (5000). Request limit is common to all users.

In [22]:
routes_df = pd.read_csv("routes2.csv")
routes_df.head(5)

,Og-data-row-number,Route-coordinates,Distance [km]
0,0,"[[(22.745049, 75.892472), (22.745074, 75.89247...",4.1606
1,1,"[[(12.913096, 77.682969), (12.911958, 77.68272...",28.9704
2,2,"[[(12.914273, 77.678365), (12.913841, 77.67824...",5.1441
3,3,"[[(11.00363, 76.975974), (11.004857, 76.975877...",13.1787
4,5,"[[(17.431655, 78.408324), (17.431691, 78.40848...",9.6564


# Final preprocessing
Final processing includes merging results from OSRM API and dataset into one and removing unused collumns (like IDs, festivals, vehicle). Then to simplify modeling meal preparation time is calculated.

# Additional preparation for model
To be able to use categorical data like traffic level or multiple deliveries we mapped non-numerical values to index variable in range 1-4 to use it in stan model. Also standarization for road distances, meal preparation and delivery person rating was made as data was to widely spread which cause problems inmodeling. This also simplified interpretation of model and its parts. 

Filtering data to manualy set maximum distance of the delivery still is a vaiable option.